# Visualize alerts!
To run this, you need to have `appsci_utils` and `deforestation` modules installed.
```
$ cd appsci_utils
$ pip install .
$ cd ~/deforestation
$ python setup develop --user
```

#This notebook has been altered to 

In [1]:
# Definet the parameter file
par_file = "par/alerts_20210815_20210901.py"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from collections import Counter
from copy import copy
import datetime
import imp
import json
import logging
logging.basicConfig(level=logging.INFO)
import os
import yaml

import geopandas as gpd
from glob import glob
import ipyleaflet
import ipywidgets
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from pathlib import Path
import shapely
from shapely import geometry
import shutil
from subprocess import check_output
from tqdm import tqdm
from typing import List, Optional

from composite_wf import composite
import descarteslabs as dl
import descarteslabs.workflows as wf

from appsci_utils.schema.coercion_functions import load_and_validate_source
from deforestation.reporting.schema import metadata_schema
from unilever.write_alerts_delivery_UL_v2 import WriteHandler

/opt/conda/lib/python3.7/site-packages/descarteslabs/common/workflows/arrow_serialization/context.py:57: FutureWarning: 'pyarrow.SerializationContext' is deprecated and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  serialization_context = pa.SerializationContext()
/opt/conda/lib/python3.7/site-packages/descarteslabs/common/workflows/arrow_serialization/context.py:58: FutureWarning: 'pyarrow.register_default_serialization_handlers' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  pa.register_default_serialization_handlers(serialization_context)


## Load alerts

In [6]:
# Copy alerts and clusters files from GCS

# Load the parameter file, which contains all of the file paths
params = load_and_validate_source(par_file, schema=metadata_schema)
handler = WriteHandler(params, "tmp")

alerts_fname = handler.output.alerts.gs.geojson.split("/")[-1]
clusters_fname = handler.output.clusters.gs.geojson.split("/")[-1]

# Local file paths
local_dir = handler.output.alerts.gs.geojson.split("/")[-2]+"/"
clusters_fpath = os.path.join(local_dir, clusters_fname)
alerts_fpath = os.path.join(local_dir, alerts_fname)

# Copy files from GCS to local
Path(local_dir).mkdir(exist_ok=True)
if not os.path.exists(os.path.join(local_dir, alerts_fname)):
    cmd = f"gsutil cp {handler.output.alerts.gs.geojson} {local_dir}"
    print(cmd)
    check_output(cmd.split())
if not os.path.exists(os.path.join(local_dir, clusters_fname)):
    cmd = f"gsutil cp {handler.output.clusters.gs.geojson} {local_dir}"
    print(cmd)
    check_output(cmd.split())
    
# Print local files
print("Local files are at:")
print(clusters_fpath)
print(alerts_fpath)

gsutil cp gs://dl-appsci/deforestation/reporting/unilever/metadata_delivery_2021-08-15_2021-09-01/alerts_delivery_2021-08-15_2021-09-01.geojson metadata_delivery_2021-08-15_2021-09-01/
gsutil cp gs://dl-appsci/deforestation/reporting/unilever/metadata_delivery_2021-08-15_2021-09-01/clusters_delivery_2021-08-15_2021-09-01.geojson metadata_delivery_2021-08-15_2021-09-01/
Local files are at:
metadata_delivery_2021-08-15_2021-09-01/clusters_delivery_2021-08-15_2021-09-01.geojson
metadata_delivery_2021-08-15_2021-09-01/alerts_delivery_2021-08-15_2021-09-01.geojson


In [7]:
# Load clusters and alerts
print(f"Open {clusters_fpath}")
clusters = gpd.read_file(clusters_fpath)
ind_c = clusters[clusters["industrial_priority"] ==1]
sm_c = clusters[clusters["smallholder_priority"] ==1]
print(f"Number of clusters: {len(clusters)}. Priority ind/small-holder = [{len(ind_c)}, {len(sm_c)}]")

print(f"Open {alerts_fpath}")
alerts = gpd.read_file(alerts_fpath)
ind_a = alerts[alerts["industrial_priority"] ==1]
sm_a = alerts[alerts["smallholder_priority"] ==1]
print(f"Number of alerts: {len(alerts)}. Priority ind/small-holder = [{len(ind_a)}, {len(sm_a)}]")

Open metadata_delivery_2021-08-15_2021-09-01/clusters_delivery_2021-08-15_2021-09-01.geojson
Number of clusters: 11099. Priority ind/small-holder = [6, 9]
Open metadata_delivery_2021-08-15_2021-09-01/alerts_delivery_2021-08-15_2021-09-01.geojson
Number of alerts: 16180. Priority ind/small-holder = [16, 714]


In [8]:
# Concessions: Get or make a directory for concessions in deforestation/data/wri_concessions/
# Available at: https://data.globalforestwatch.org/datasets/d7d79dee63124deaaad070cc9d820ade_0
try:
    cc_dir = Path(Path(os.getcwd()).parent, "data", "wri_concessions")
    cc_dir.mkdir(parents=True, exist_ok=True)
    cc_src = "gs://dl-appsci/deforestation/data/wri_concessions/*.zip"

    # Download concessions data from GCS, unpack
    if any([
        not Path(cc_dir, "Indonesia_oil_palm_concessions.shp").exists(),
        not Path(cc_dir, "Sarawak_oil_palm_concessions.shp").exists(),
        not Path(cc_dir, "Indonesia_wood_fiber_concessions.shp").exists(),
        not Path(cc_dir, "Sarawak_logging_concessions.shp").exists(),
        not Path(cc_dir, "Sarawak_Licenses_for_Planted_Forests_(LPFs).shp").exists(),
    ]):
        cmd = f"gsutil -m cp {cc_src} {cc_dir}/"
        print(cmd)
        check_output(cmd.split())
        _ = [shutil.unpack_archive(ff, cc_dir) for ff in glob(os.path.join(cc_dir, "*.zip"))]

    # Load concessions into 2 files, palm and wood
    i_palm_concessions = gpd.read_file("../data/wri_concessions/Indonesia_oil_palm_concessions.shp")
    s_palm_concessions = gpd.read_file("../data/wri_concessions/Sarawak_oil_palm_concessions.shp")
    palm_concessions = i_palm_concessions.append(s_palm_concessions)
    print(f"Loaded palm concessions with {len(palm_concessions)} polygons")

    i_wood_concessions = gpd.read_file("../data/wri_concessions/Indonesia_wood_fiber_concessions.shp")
    s_wood_concessions = gpd.read_file("../data/wri_concessions/Sarawak_logging_concessions.shp")
    s_lpf_concessions = gpd.read_file("../data/wri_concessions/Sarawak_Licenses_for_Planted_Forests_(LPFs).shp")
    wood_concessions = i_wood_concessions.append(s_wood_concessions)
    wood_concessions = wood_concessions.append(s_lpf_concessions)
    print(f"Loaded wood concessions with {len(wood_concessions)} polygons")
except:
    print("Couldn't load concessions data")

Loaded palm concessions with 2093 polygons
Loaded wood concessions with 863 polygons


In [9]:
# Make an area-weighted heat map of alerts
loc_area = []
for idx, alert in tqdm(alerts.iterrows()):
    center = alert.geometry.centroid.xy
    lon, lat = center[0][0], center[1][0]
    loc_area.append([lat, lon, alert.alert_area])
    
# Area-weighted heatmap
print("Make HeatMap")
heatmap2 = ipyleaflet.Heatmap(
    locations = loc_area,
    radius=5,
    blur=4,
    max=0.05,
    gradient={0.2: "blue", 0.3: "cyan", 0.7: "purple", 0.9: "yellow", 1.: "red"},
)

16180it [00:02, 6280.40it/s]


Make HeatMap


In [10]:
# Define Before - After composites
kwargs = {
    "product": "sentinel-2:L1C",
    "bands": ["red", "green", "blue"],
    "processing_level": "surface",
    "resampler": "near",
   "strategy": "median",    
    "cloudmask_product": "sentinel-2:L1C:dlcloud:v1",
    "cloudmask_invalid": 0.1,
    "cloudmask_bands": ["valid_cloudfree"],
    "imagemask_bands": ["bright-mask", "cloud-mask"],
}
kwargs_before = copy(kwargs)
kwargs_before["start_datetime"] = "2020-01-01"
kwargs_before["end_datetime"] = "2020-02-01"

kwargs_after = copy(kwargs)
kwargs_after["start_datetime"] = "2021-01-01"
kwargs_after["end_datetime"] = "2021-02-01"

In [ ]:
# Vector layers
clusters_layer = ipyleaflet.GeoData(
    geo_dataframe = clusters,
    style={"color": "red", "fillOpacity":0.},
    name="clusters",
)
alerts_layer = ipyleaflet.GeoData(
    geo_dataframe = alerts,
    name='alerts',
    style={'color': 'yellow', 'fillOpacity':0.},
)

# Concessions
try:
    palm_concessions_layer = ipyleaflet.GeoData(
            geo_dataframe = palm_concessions,
            name='palm_concessions',
            style={'color': 'blue', 'fillOpacity':0},
    )
    wood_concessions_layer = ipyleaflet.GeoData(
            geo_dataframe = wood_concessions,
            name='wood_concessions',
            style={'color': 'purple', 'fillOpacity':0},
    )
except:
    print("Failed to load concessions")

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [ ]:
# Raster layers
s2_2020 = (
    wf.ImageCollection.from_id("descarteslabs:forest_carbon:S2_GEDI:composite:v1.0",
                               start_datetime="2020-06-30",
                               end_datetime="2020-07-02")
    .pick_bands(["red", "green", "blue"])
    .mosaic()
)

# Before / After S2 composites
# before_c = composite(**kwargs_before).pick_bands(kwargs["bands"])
# after_c = composite(**kwargs_after).pick_bands(kwargs["bands"])

# Deforestation
def_v3 = (
    wf.ImageCollection.from_id("descarteslabs:ul_deforestation_external_v3",
                               start_datetime="2020-01-01",
                               end_datetime="2022-01-01")
    .pick_bands(["detection_date"])
    .sorted(lambda img: img.properties['date'])
    .mosaic()
)
def_hist = (
    wf.ImageCollection.from_id("descarteslabs:ul_deforestation_combined_v3",
                               start_datetime="2017-01-01",
                               end_datetime="2021-01-01")
    .pick_bands(["detection_date"])
    .mosaic()
)

# Combine historical and real-time deforestation products,
# preferring the real-time alerts where they overlap
deforestation = wf.concat(def_hist, def_v3).mosaic()
# historical_only = def_hist.mask(~def_v3.getmask())

# Scales for visualization
start_int = int((np.datetime64("2018-01-01") - np.datetime64("2015-01-01")) / np.timedelta64(1, "D"))
end_int = int((np.datetime64(params["end_datetime"]) - np.datetime64("2015-01-01")) / np.timedelta64(1, "D"))
print(start_int, end_int, params["end_datetime"])

# Visualize

In [ ]:
# Create the map
m1 = None
m1 = wf.interactive.MapApp()
#m1.center = (0.6056, 116.4798) # Borneo
#m1.center = (3.0689, 97.5589) # Aceh
#m1.zoom = 12
m1.add_control(ipyleaflet.LayersControl(position='topright'))
m1.control_other_layers = True
m1.map.layout.height = "600px"

In [ ]:
# Add on_click handler
# Pop-up window for feature properties
def popup_m1(feature, **kwargs):
    geom = geometry.shape(feature["geometry"])
    popup = ipyleaflet.Popup(
        location=[geom.centroid.y, geom.centroid.x],
        child=ipywidgets.HTML(value=f"{feature['properties']}"),
        close_button=True,
        auto_close=True,
        name="popup_properties"
    )
    try:
        m1.remove_layer("popup_properties")
    except:
        pass
    m1.add_layer(popup)
    
clusters_layer.on_click(popup_m1)
try:
    palm_concessions_layer.on_click(popup_m1)
    wood_concessions_layer.on_click(popup_m1)
except:
    print("Failed to load concessions")

In [ ]:
m1.clear_layers()

# Add layers in the order I want to turn them on
s2_2020.visualize(name="Sentintel 2", map=m1, scales=[[0,1200], [0,1200], [0,1200]], checkerboard=False)
#before_c.visualize("before", scales=[[0, 2000], [0, 2000], [0, 2000]], map=m1)
#after_c.visualize("after", scales=[[0, 2000], [0, 2000], [0, 2000]], map=m1)

# For carbon, pixel values are in [MgC/pixel]. 27 MgC/pixel = 300 MgC/ha = 27*1e4/(30**2)
# agc.visualize(name="above_ground_carbon", map=m1, colormap="viridis", scales=[[0, 27]], checkerboard=False)
deforestation.visualize(name="DL Alerts", colormap="plasma",scales=[[start_int, end_int]], map=m1, checkerboard=False)
#def_hist.visualize(name="DL 2020", map=m1, checkerboard=False)

try:
   m1.add_layer(palm_concessions_layer)
   m1.add_layer(wood_concessions_layer)
except:
   print("Failed to load concessions")


m1.add_layer(alerts_layer)
m1.add_layer(clusters_layer)
m1.add_layer(heatmap2)

In [ ]:
# RADD Deforestation
radd = (
    wf.ImageCollection.from_id("descarteslabs:wri_radd_alerts:v1",
                               start_datetime="2020-01-01",
                               end_datetime="2021-07-01")
#        .pick_bands(["date", "alert"])
#    .mosaic()
)
alert_mask = radd.pick_bands(["date"])
radd = radd.pick_bands(["alert"])

radd_mask = radd != 3
alert_mask= alert_mask <= 20001

radd = radd.mask(radd_mask)
radd = radd.mask(alert_mask).mosaic()

radd.visualize(name="RADD alerts 2020 to July 21", map=m1, checkerboard=False)



In [ ]:
m1

In [ ]:
# m1.center = (0.8789, 106.5234) #Map overview
# m1.zoom = 6

# m1.center =(3.9483, 96.7275) # Aceh
# m1.zoom = 13

# m1.center =(0.3529, 102.8226) # Pelawan/Riau
# m1.zoom = 13

#m1.center =(-2.1895, 103.1618) # South Sumatra/Musi Banyuasin
#m1.zoom = 13

m1.center =(1.1827, 109.7441) # Sambas
m1.zoom = 13


In [ ]:

deforestation.visualize(name="DL Historical", map=m1, colormap="plasma", scales=[[start_int, end_int]], checkerboard=False)